# DSC540 - Final Term Project Milestone 4 - Connecting to an API/Pulling in the Data and Cleaning/Formatting

## Janine Par 

### For this milestone, I will be working on two API (s):

1. Us-Cities Demographic information by Opendatasoft

2. Yelp API - Business Reviews 

### Import necessary libraries including request, json, panda, etc

In [23]:
import pandas as pd
import requests
import re

import urllib.request
import urllib.request, urllib.parse
from urllib.error import HTTPError,URLError
import json

# US DEMOGRAPHIC INFORMATION API

### Call the API using urllib,request
#### Step1: Create a function that will accept URL as parameter. This function will check connection and if successful, it will read and decode and return the response

In [2]:
def get_data(url):
 
    try: 
        uh = urllib.request.urlopen(url)
    except HTTPError as e:
        print("Sorry! Could not retrive anything on")
        return None
    except URLError as e:
        print('Failed to reach a server.')
        print('Reason: ', e.reason)
        return None
    else:
        data = uh.read().decode()
        print("Sucess")
        #rint("Retrieved data on {}. Total {} characters read.".format(country_name,len(data)))
        return data

In [3]:
url_uscities = "https://documentation-resources.opendatasoft.com/api/records/1.0/search/?dataset=us-cities-demographics&q=&rows=2891&facet=city&facet=state"

In [4]:
data=get_data(url_uscities)

Sucess


### Step2: Load ther result to a dictionary 

In [5]:
# Load from string 'data'
us_cities_dem=json.loads(data)
type(us_cities_dem)

dict

### Step3: Determine the keys to identify the columns needed in the dataframe 

In [6]:
us_cities_dem.keys()

dict_keys(['nhits', 'parameters', 'records', 'facet_groups'])

In [7]:
# record information are inside key = Records, determine the columns needed inside
us_cities_dem['records'][1].keys()

dict_keys(['datasetid', 'recordid', 'fields', 'record_timestamp'])

In [10]:
# field information are inside key = fields, determine the columns needed inside
us_cities_dem['records'][1]['fields'].keys()

dict_keys(['male_population', 'city', 'number_of_veterans', 'median_age', 'race', 'count', 'female_population', 'state_code', 'foreign_born', 'state', 'total_population', 'average_household_size'])

#### Only geting the 'records' section of the response since only field information from this section is needed 

In [11]:
us_cities_rec=us_cities_dem['records']
us_cities_rec[:10]

[{'datasetid': 'us-cities-demographics',
  'recordid': '9ff2cdb9e7cbe1585135f848ab90d63a0934a2da',
  'fields': {'male_population': 40601,
   'city': 'Silver Spring',
   'number_of_veterans': 1562,
   'median_age': 33.8,
   'race': 'Hispanic or Latino',
   'count': 25924,
   'female_population': 41862,
   'state_code': 'MD',
   'foreign_born': 30908,
   'state': 'Maryland',
   'total_population': 82463,
   'average_household_size': 2.6},
  'record_timestamp': '2020-12-30T13:35:36.384Z'},
 {'datasetid': 'us-cities-demographics',
  'recordid': 'dd9bab63ddc55c438457d919dca8073d5268b791',
  'fields': {'male_population': 51751,
   'city': 'High Point',
   'number_of_veterans': 5204,
   'median_age': 35.5,
   'race': 'Asian',
   'count': 11060,
   'female_population': 58077,
   'state_code': 'NC',
   'foreign_born': 16315,
   'state': 'North Carolina',
   'total_population': 109828,
   'average_household_size': 2.65},
  'record_timestamp': '2020-12-30T13:35:36.384Z'},
 {'datasetid': 'us-citie

### Step4: 
* Create a blank dictionary with list of each fields identified from Step3

In [12]:
#First get the dataset and record_id and create list
data_rec_dtl={'datasetid':[],'recordid':[],'record_timestamp':[] ,'male_population':[],'city':[],'number_of_veterans':[],
             'median_age':[],'race':[],'count':[] ,'female_population':[],'state_code':[],'foreign_born':[],
             'state':[],'total_population':[],'average_household_size':[] }


 * Then loop through the us_cities_dem list to capture each data-element information and then append to the final list

In [13]:
#Some fields are seem to be dynamic where some keys are not in the record so I used dict.get to avoid key value error
for i, dtl in enumerate(us_cities_rec):
    data_rec_dtl['record_timestamp'].append(us_cities_rec[i]['record_timestamp'])
    data_rec_dtl['datasetid'].append(us_cities_rec[i]['datasetid'])
    data_rec_dtl['recordid'].append(us_cities_rec[i]['recordid'])  
    data_rec_dtl['male_population'].append(us_cities_rec[i]['fields'].get('male_population','none'))
    data_rec_dtl['city'].append(us_cities_rec[i]['fields'].get('city',0))
    data_rec_dtl['number_of_veterans'].append(us_cities_rec[i]['fields'].get('number_of_veterans',0))
        
    data_rec_dtl['median_age'].append(us_cities_rec[i]['fields'].get('median_age'))
    data_rec_dtl['race'].append(us_cities_rec[i]['fields'].get('race'))
    data_rec_dtl['count'].append(us_cities_rec[i]['fields'].get('count'))
    
    data_rec_dtl['female_population'].append(us_cities_rec[i]['fields'].get('female_population'))
    data_rec_dtl['foreign_born'].append(us_cities_rec[i]['fields'].get('foreign_born'))
    data_rec_dtl['state_code'].append(us_cities_rec[i]['fields'].get('state_code'))
    
    
    data_rec_dtl['state'].append(us_cities_rec[i]['fields'].get('state'))
    data_rec_dtl['total_population'].append(us_cities_rec[i]['fields'].get('total_population'))
    data_rec_dtl['average_household_size'].append(us_cities_rec[i]['fields'].get('average_household_size'))
    

#### Verifying if count is the same accross the lists

In [14]:
# Check if all dictionary have same length 

for k in data_rec_dtl:
    print(k+' - '+str(len(data_rec_dtl[k])))
    


datasetid - 2891
recordid - 2891
record_timestamp - 2891
male_population - 2891
city - 2891
number_of_veterans - 2891
median_age - 2891
race - 2891
count - 2891
female_population - 2891
state_code - 2891
foreign_born - 2891
state - 2891
total_population - 2891
average_household_size - 2891


### Step4: Create dataframe of the US CITIES DEMOGRAPHIC

In [24]:
df_us_cities_dem = pd.DataFrame(data_rec_dtl)
df_us_cities_dem.shape

(1500, 5)

In [16]:
df_us_cities_dem.head()

,datasetid,recordid,record_timestamp,male_population,city,number_of_veterans,median_age,race,count,female_population,state_code,foreign_born,state,total_population,average_household_size
0,us-cities-demographics,9ff2cdb9e7cbe1585135f848ab90d63a0934a2da,2020-12-30T13:35:36.384Z,40601,Silver Spring,1562,33.8,Hispanic or Latino,25924,41862.0,MD,30908.0,Maryland,82463,2.60
1,us-cities-demographics,dd9bab63ddc55c438457d919dca8073d5268b791,2020-12-30T13:35:36.384Z,51751,High Point,5204,35.5,Asian,11060,58077.0,NC,16315.0,North Carolina,109828,2.65
2,us-cities-demographics,413ed05fec8b9450e35ac3010ddef17bcfbd83a6,2020-12-30T13:35:36.384Z,741270,Philadelphia,61995,34.1,Asian,122721,826172.0,PA,205339.0,Pennsylvania,1567442,2.61
3,us-cities-demographics,0074451cff52969855654d21497e9459f1108d8d,2020-12-30T13:35:36.384Z,192354,Wichita,23978,34.6,American Indian and Alaska Native,8791,197601.0,KS,40270.0,Kansas,389955,2.56
4,us-cities-demographics,54b201cac9c7523363eb0cfeadc352a04fe016af,2020-12-30T13:35:36.384Z,60626,Allen,5691,33.5,Black or African-American,22304,59581.0,PA,19652.0,Pennsylvania,120207,2.67


### Step5: Examine missing values and drop na (threshold =10)

In [17]:
def df_missing_cols (df):
    for c in df.columns:
            is_missing=df[c].isna().sum()
            if is_missing > 0:
                print ("{} has {} missing values (s)". format (c,is_missing))
            else:
                print ("{} has no missing values".format(c))

In [18]:
def drop_nan (df):
    df=df.dropna(thresh=10)
    return(df)

In [19]:
df_missing_cols(df_us_cities_dem)

datasetid has no missing values
recordid has no missing values
record_timestamp has no missing values
male_population has no missing values
city has no missing values
number_of_veterans has no missing values
median_age has no missing values
race has no missing values
count has no missing values
female_population has 3 missing values (s)
state_code has no missing values
foreign_born has 13 missing values (s)
state has no missing values
total_population has no missing values
average_household_size has 16 missing values (s)


In [20]:
df_us_cities_dem=drop_nan(df_us_cities_dem)
df_us_cities_dem.shape

(2891, 15)

### Create Flat File for database load

### Step6: Do some string clean-up
* Remove whitespace, make statecode upper case and remove if there are any uneccessary characters

In [22]:
#upper case country an remove unwanted characters
df_us_cities_dem["state_code"] = df_us_cities_dem["state_code"].str.strip().str.upper().str.replace("/\A[a-z0-9\s]+\Z/i",'')


C:\Users\janin\AppData\Local\Temp\ipykernel_10152\573229857.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df_us_cities_dem["state_code"] = df_us_cities_dem["state_code"].str.strip().str.upper().str.replace("/\A[a-z0-9\s]+\Z/i",'')


* Make race all lower case then validate

In [25]:
df_us_cities_dem["race"] = df_us_cities_dem["race"].str.lower()
df_us_cities_dem["race"].value_counts()

hispanic or latino                   596
white                                589
black or african-american            584
asian                                583
american indian and alaska native    539
Name: race, dtype: int64

In [24]:
df_us_cities_dem.head()

,datasetid,recordid,record_timestamp,male_population,city,number_of_veterans,median_age,race,count,female_population,state_code,foreign_born,state,total_population,average_household_size
0,us-cities-demographics,9ff2cdb9e7cbe1585135f848ab90d63a0934a2da,2020-12-30T13:35:36.384Z,40601,Silver Spring,1562,33.8,hispanic or latino,25924,41862.0,MD,30908.0,Maryland,82463,2.60
1,us-cities-demographics,dd9bab63ddc55c438457d919dca8073d5268b791,2020-12-30T13:35:36.384Z,51751,High Point,5204,35.5,asian,11060,58077.0,NC,16315.0,North Carolina,109828,2.65
2,us-cities-demographics,413ed05fec8b9450e35ac3010ddef17bcfbd83a6,2020-12-30T13:35:36.384Z,741270,Philadelphia,61995,34.1,asian,122721,826172.0,PA,205339.0,Pennsylvania,1567442,2.61
3,us-cities-demographics,0074451cff52969855654d21497e9459f1108d8d,2020-12-30T13:35:36.384Z,192354,Wichita,23978,34.6,american indian and alaska native,8791,197601.0,KS,40270.0,Kansas,389955,2.56
4,us-cities-demographics,54b201cac9c7523363eb0cfeadc352a04fe016af,2020-12-30T13:35:36.384Z,60626,Allen,5691,33.5,black or african-american,22304,59581.0,PA,19652.0,Pennsylvania,120207,2.67


In [81]:
df_us_cities_dem.to_csv('df_us_cities_dem_m4.txt', sep='|', index=False)


# Yelp Business Reviews 
This API returns max of 3 reviews and details for a given business ID

### Step1: This requires API thus upload the API Key from the APIKeys.json file

In [3]:
with open('APIkeys.json') as f:
    keys = json.load(f)
    Yelpapi = keys['Yelpapi']

### Step2: Call the API using request get 


In [4]:
# API constants, you shouldn't have to change these.
YelpHost = 'https://api.yelp.com/v3/businesses/'


#### Create a function that will accept host id, business ID, apikey and url paramter needed when calling API

In [5]:
def request(host, busid, api_key, url_params=None):
    """Given your API_KEY, send a GET request to the API.
    Args:
        host (str): The domain host of the API.
        path (str): The path of the API after the domain.
        API_KEY (str): Your API Key.
        url_params (dict): An optional set of query parameters in the request.
    Returns:
        dict: The JSON response from the request.
    Raises:
        HTTPError: An error occurs from the HTTP request.
    """
    url_params = url_params or {}
    url = ('{0}{1}/reviews?limit=1&sort_by=yelp_sort'.format(host, busid))
    headers = {'Authorization': 'Bearer %s' % api_key, "accept": "application/json"}

    response = requests.request('GET', url, headers=headers, params=url_params)

    return response.json()


#### Since business ID is needed for this, I have used the file from milestone 2 with the business detail information and use that to get the business id

In [25]:
bus = pd.read_json("yelp_academic_dataset_business.json", lines=True)


In [7]:
bus.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,CA,93101,34.426679,-119.711197,5.0,7,0,{'ByAppointmentOnly': 'True'},"Doctors, Traditional Chinese Medicine, Naturop...",None
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,MO,63123,38.551126,-90.335695,3.0,15,1,{'BusinessAcceptsCreditCards': 'True'},"Shipping Centers, Local Services, Notaries, Ma...","{'Monday': '0:0-0:0', 'Tuesday': '8:0-18:30', ..."
2,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,AZ,85711,32.223236,-110.880452,3.5,22,0,"{'BikeParking': 'True', 'BusinessAcceptsCredit...","Department Stores, Shopping, Fashion, Home & G...","{'Monday': '8:0-22:0', 'Tuesday': '8:0-22:0', ..."
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ..."
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,PA,18054,40.338183,-75.471659,4.5,13,1,"{'BusinessAcceptsCreditCards': 'True', 'Wheelc...","Brewpubs, Breweries, Food","{'Wednesday': '14:0-22:0', 'Thursday': '16:0-2..."


In [26]:
bus_rest = bus.query('categories.str.contains("Restaurants") or categories.str.contains("Food")', engine='python')


In [27]:
bus_rest=bus_rest.reset_index(drop='TRUE')

In [28]:
bus_rest.to_csv('bus_rest_merge_m2.txt', sep='|', index=False)

#### 1. For this example, I'm just taking the 100 business ID from the file 
#### 2. I looped through the lenght of business ID and get the business ID from the bus dataframe then call the API request for each business ID. 
#### 3. I appended each response to a list  response_list then check length of the list 


In [10]:
len(bus_rest)

64616

In [11]:
response_list=[]

for i in range (500): #Only has limit of 500
    businessId=bus_rest['business_id'][i]
    response=request(YelpHost,businessId, Yelpapi) #Call the request
    response['business_id']=businessId #add business id to response dictionary
    response_list.append(response)

len(response_list)




500

In [13]:
# Check 1o responses in the list 
response_list[:1]

[{'reviews': [{'id': '2l02e-2QAYZnF0tbZz5a-A',
    'url': 'https://www.yelp.com/biz/st-honore-pastries-philadelphia?adjust_creative=2YkLfb-An6EFA8djH8GpBA&hrid=2l02e-2QAYZnF0tbZz5a-A&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=2YkLfb-An6EFA8djH8GpBA',
    'text': "Made a quick trip to Philly to get some eats! Of course, knew we'd have to pick up some of the paper wrapped sponge cakes from St. Honore. We called ahead...",
    'rating': 5,
    'time_created': '2022-07-25 09:43:43',
    'user': {'id': 'CKBLVYZi8len88s1meKIoA',
     'profile_url': 'https://www.yelp.com/user_details?userid=CKBLVYZi8len88s1meKIoA',
     'image_url': 'https://s3-media1.fl.yelpcdn.com/photo/fa8I6yUxyp6B4AOR1ABGLA/o.jpg',
     'name': 'Jo M.'}},
   {'id': 'hr4C7vsahxkieDJ9tqtm0A',
    'url': 'https://www.yelp.com/biz/st-honore-pastries-philadelphia?adjust_creative=2YkLfb-An6EFA8djH8GpBA&hrid=hr4C7vsahxkieDJ9tqtm0A&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=

### Step3: Read each of the response in the list and capture information to create dataset 


#### The response has the Key:total review which is the sum count of each business, also Key: Reviews that has dictionary with review field information. There can be max of 3 Key:reviews for each business ID

I have created two blank dictionares:
* data_rec_sum will have the business ID and total review
* data_rec_dtl will have the multiple reviews for each business ID

In [14]:
data_rec_sum={'business_id':[],'total_review':[]}
data_rec_dtl={'business_id':[],'review_id':[],'url':[] ,'text':[],'rating':[]}


#### Create function that will get the business review details with business and the index of the list 

* Get the count of the review for each business id
* Loop through iterate based on the number of review of each business 
* Get each review field information and append to the list 

**This get_response_field will be called on the next step "Let's build the dictionary "

In [15]:
def get_response_field(business_id, idx):

    range_review=len(response_list[idx]['reviews']) # get the number of reviews for a business 

    for x in range(0,range_review):
        #'id', 'url', 'text', 'rating', 'time_created' 'user'
        data_rec_dtl['business_id'].append(business_id)
        data_rec_dtl['review_id'].append(response_list[idx]['reviews'][x]['id'])
        data_rec_dtl['url'].append(response_list[idx]['reviews'][x]['url'])
        data_rec_dtl['text'].append(response_list[idx]['reviews'][x]['text'])
        data_rec_dtl['rating'].append(response_list[idx]['reviews'][x]['rating'])
                  

#### Let's build the dictionary 
* Loop through the response list and for each business ID
  * Get the business ID and Total Review and create the data_rec_sum dictionary 
  * Call the ***get response field function (from above step) to get the field information  and create the data_rec_dtl dictionary

In [16]:
for idx,x in enumerate(response_list):
    data_rec_sum['business_id'].append(response_list[idx]['business_id'])
    data_rec_sum['total_review'].append(response_list[idx]['total'])
    
    # Call the get response field to gather review field information
    get_response_field((response_list[idx]['business_id']),idx) 

### Step4 Review results

In [17]:
df_review_sum= pd.DataFrame(data_rec_sum)
df_review_sum.head()

,business_id,total_review
0,MTSW4McQd7CbVtyjqoe9mw,82
1,mWMc6_wTdE0EUBKIGXDVfA,15
2,CF33F8-E6oudUQ46HnavjQ,11
3,k0hlBqXX-Bt0vf1op7Jr1w,20
4,bBDDEgkFA1Otx9Lfe7BZUQ,16


In [18]:
df_review_dtl= pd.DataFrame(data_rec_dtl)
df_review_dtl.head()

,business_id,review_id,url,text,rating
0,MTSW4McQd7CbVtyjqoe9mw,2l02e-2QAYZnF0tbZz5a-A,https://www.yelp.com/biz/st-honore-pastries-ph...,Made a quick trip to Philly to get some eats! ...,5
1,MTSW4McQd7CbVtyjqoe9mw,hr4C7vsahxkieDJ9tqtm0A,https://www.yelp.com/biz/st-honore-pastries-ph...,The crispy roast pork is SO GOOD and lowkey it...,5
2,MTSW4McQd7CbVtyjqoe9mw,khVt8RKpraoAwJg_fMjhIw,https://www.yelp.com/biz/st-honore-pastries-ph...,The breads were SUPER SOFT. The egg custard in...,5
3,mWMc6_wTdE0EUBKIGXDVfA,04dNsXLC5HvhvuHCc_rvvw,https://www.yelp.com/biz/perkiomen-valley-brew...,This place is so cool! You can tell it used to...,5
4,mWMc6_wTdE0EUBKIGXDVfA,Grfe-Rl7jbIBMchWTdZaug,https://www.yelp.com/biz/perkiomen-valley-brew...,This is our go to brewery. They have a great s...,5


### Step5: Merge the two dataframe on business id using inner join

In [19]:
df_review = pd.merge(df_review_sum,df_review_dtl , on='business_id', how = 'inner')

In [20]:
df_review.head()

,business_id,total_review,review_id,url,text,rating
0,MTSW4McQd7CbVtyjqoe9mw,82,2l02e-2QAYZnF0tbZz5a-A,https://www.yelp.com/biz/st-honore-pastries-ph...,Made a quick trip to Philly to get some eats! ...,5
1,MTSW4McQd7CbVtyjqoe9mw,82,hr4C7vsahxkieDJ9tqtm0A,https://www.yelp.com/biz/st-honore-pastries-ph...,The crispy roast pork is SO GOOD and lowkey it...,5
2,MTSW4McQd7CbVtyjqoe9mw,82,khVt8RKpraoAwJg_fMjhIw,https://www.yelp.com/biz/st-honore-pastries-ph...,The breads were SUPER SOFT. The egg custard in...,5
3,mWMc6_wTdE0EUBKIGXDVfA,15,04dNsXLC5HvhvuHCc_rvvw,https://www.yelp.com/biz/perkiomen-valley-brew...,This place is so cool! You can tell it used to...,5
4,mWMc6_wTdE0EUBKIGXDVfA,15,Grfe-Rl7jbIBMchWTdZaug,https://www.yelp.com/biz/perkiomen-valley-brew...,This is our go to brewery. They have a great s...,5


### Store in Separate file for DB loading

In [22]:
df_review.to_csv('df_review_m4.txt', sep='|', index=False)